In [310]:
import pandas as pd

In [311]:
df = pd.read_csv("laptop_price.csv", encoding="latin-1")

In [312]:
df

laptop_ID Company                              Product  \
0             1   Apple                          MacBook Pro   
1             2   Apple                          Macbook Air   
2             3      HP                               250 G6   
3             4   Apple                          MacBook Pro   
4             5   Apple                          MacBook Pro   
...         ...     ...                                  ...   
1298       1316  Lenovo                       Yoga 500-14ISK   
1299       1317  Lenovo                       Yoga 900-13ISK   
1300       1318  Lenovo                   IdeaPad 100S-14IBR   
1301       1319      HP  15-AC110nv (i7-6500U/6GB/1TB/Radeon   
1302       1320    Asus  X553SA-XX031T (N3050/4GB/500GB/W10)   

                TypeName  Inches                            ScreenResolution  \
0              Ultrabook    13.3          IPS Panel Retina Display 2560x1600   
1              Ultrabook    13.3                                    1440x900   
2               Notebook    15.6                           Full HD 1920x1080   
3              Ultrabook    15.4          IPS Panel Retina Display 2880x1800   
4              Ultrabook    13.3          IPS Panel Retina Display 2560x1600   
...                  ...     ...                                         ...   
1298  2 in 1 Convertible    14.0   IPS Panel Full HD / Touchscreen 1920x1080   
1299  2 in 1 Convertible    13.3  IPS Panel Quad HD+ / Touchscreen 3200x1800   
1300            Notebook    14.0                                    1366x768   
1301            Notebook    15.6                                    1366x768   
1302            Notebook    15.6                                    1366x768   

                                       Cpu   Ram               Memory  \
0                     Intel Core i5 2.3GHz   8GB            128GB SSD   
1                     Intel Core i5 1.8GHz   8GB  128GB Flash Storage   
2               Intel Core i5 7200U 2.5GHz   8GB            256GB SSD   
3                     Intel Core i7 2.7GHz  16GB            512GB SSD   
4                     Intel Core i5 3.1GHz   8GB            256GB SSD   
...                                    ...   ...                  ...   
1298            Intel Core i7 6500U 2.5GHz   4GB            128GB SSD   
1299            Intel Core i7 6500U 2.5GHz  16GB            512GB SSD   
1300  Intel Celeron Dual Core N3050 1.6GHz   2GB   64GB Flash Storage   
1301            Intel Core i7 6500U 2.5GHz   6GB              1TB HDD   
1302  Intel Celeron Dual Core N3050 1.6GHz   4GB            500GB HDD   

                               Gpu       OpSys  Weight  Price_euros  
0     Intel Iris Plus Graphics 640       macOS  1.37kg      1339.69  
1           Intel HD Graphics 6000       macOS  1.34kg       898.94  
2            Intel HD Graphics 620       No OS  1.86kg       575.00  
3               AMD Radeon Pro 455       macOS  1.83kg      2537.45  
4     Intel Iris Plus Graphics 650       macOS  1.37kg      1803.60  
...                            ...         ...     ...          ...  
1298         Intel HD Graphics 520  Windows 10   1.8kg       638.00  
1299         Intel HD Graphics 520  Windows 10   1.3kg      1499.00  
1300             Intel HD Graphics  Windows 10   1.5kg       229.00  
1301            AMD Radeon R5 M330  Windows 10  2.19kg       764.00  
1302             Intel HD Graphics  Windows 10   2.2kg       369.00  

[1303 rows x 13 columns]

In [313]:
df = df.drop("Product", axis=1)

In [314]:
df = df.join(pd.get_dummies(df.Company))
df = df.drop("Company", axis=1)

In [315]:
df = df.join(pd.get_dummies(df.TypeName))
df = df.drop("TypeName", axis=1)

In [316]:
df["ScreenResolution"] = df.ScreenResolution.str.split(" ").apply(lambda x: x[-1])
df["Screen Width"] = df.ScreenResolution.str.split("x").apply(lambda x: x[0])
df["Screen Height"] = df.ScreenResolution.str.split("x").apply(lambda x: x[1])

                                                                  

In [317]:
df = df.drop("ScreenResolution", axis=1)

In [318]:
df["CPU Brand"] = df.Cpu.str.split(" ").apply(lambda x: x[0])
df["CPU Frequency"] = df.Cpu.str.split(" ").apply(lambda x: x[-1])

In [319]:
df = df.drop("Cpu", axis=1)

In [320]:
df["CPU Frequency"] = df["CPU Frequency"].str[:-3]

In [321]:
df["Ram"] = df["Ram"].str[:-2]

In [322]:
df["Ram"] = df["Ram"].astype("int")
df["CPU Frequency"] = df["CPU Frequency"].astype("float")

In [323]:
df["Screen Width"] = df["Screen Width"].astype("int")
df["Screen Height"] = df["Screen Height"].astype("int")

In [324]:
df["Memory Amount"] = df.Memory.str.split(" ").apply(lambda x: x[0])
df["Memory Type"] = df.Memory.str.split(" ").apply(lambda x: x[1])

In [325]:
def turn_memory_into_MB(value):
    if "GB" in value:
        return float(value[:value.find("GB")]) * 1000
    elif "TB" in value:
        return float(value[:value.find("TB")]) * 1000000    

In [326]:
df["Memory Amount"] = df["Memory Amount"].apply(turn_memory_into_MB)

In [327]:
df = df.drop("Memory", axis=1)

In [328]:
df["Weight"] = df["Weight"].str[:-2]

In [329]:
df["Weight"] = df["Weight"].astype("float")

In [330]:
df["GPU Brand"] = df.Gpu.str.split(" ").apply(lambda x: x[0])

In [331]:
df = df.drop("Gpu", axis=1)

In [332]:
df = df.join(pd.get_dummies(df.OpSys))
df = df.drop("OpSys", axis=1)

In [333]:
cpu_categories = pd.get_dummies(df["CPU Brand"])
cpu_categories.columns = [col + "_CPU" for col in cpu_categories.columns]

df = df.join(cpu_categories)
df = df.drop("CPU Brand", axis=1)

In [334]:
gpu_categories = pd.get_dummies(df["GPU Brand"])
gpu_categories.columns = [col + "_GPU" for col in gpu_categories.columns]

df = df.join(gpu_categories)
df = df.drop("GPU Brand", axis=1)

In [335]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize(18,15))
sns.heatmap(df.corr(), annot=True, cmap="YlGnBu")

NameError: name 'figsize' is not defined

In [336]:
target_correlations = df.corr()['Price_euros'].apply(abs).sort_values()

ValueError: could not convert string to float: 'SSD'

In [337]:
selected_features = target_correlations[-21:].index

NameError: name 'target_correlations' is not defined

In [338]:
selected_features = list(selected_features)

NameError: name 'selected_features' is not defined

In [339]:
limited_df = df[selected_features]

NameError: name 'selected_features' is not defined

In [344]:
plt.figure(figsize=(18,15))
sns.heatmap(limited_df.corr(), annot=True, cmap = "YlGnBu")

NameError: name 'limited_df' is not defined

<Figure size 1800x1500 with 0 Axes>

In [347]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X,y - limited_df.drop("Price_euros", axis=1), limited_df["Price_euros"]

X_train, X_test, y_train, y_test = traint_test_split(X, y, test_size=0.15)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

NameError: name 'X' is not defined

In [348]:
forest = RandomForestRegressor()

forest.fit(X_train_scaled, y_train)

NameError: name 'X_train_scaled' is not defined

In [349]:
forest.score(X_test_scaled, y_test)

NameError: name 'X_test_scaled' is not defined

In [350]:
y_pred = forest.predict(X_test_scaled)

NameError: name 'X_test_scaled' is not defined

In [351]:
plt.figure(figsize=(12, 8))
plt.scatter(y_pred, y_test)
plt.plot(range(0,6000), range(0,6000), c="red")

NameError: name 'y_pred' is not defined

<Figure size 1200x800 with 0 Axes>

NameError: name 'X_test' is not defined

In [353]:
X_new_scaled = scaler.transform(X_test.iloc[0])

NameError: name 'scaler' is not defined

In [354]:
forest.predict(X_new_scaled)

NameError: name 'X_new_scaled' is not defined

In [ ]:
Y_test.iloc[0]